In [ ]:
import pandas as pd
import os

files_path = 'G:/Yehmh/DNDF/tree_survey_coord'
coord_file_path = 'G:/Yehmh/DNDF/DNDF_PlantLine_twd97.csv'
name_file_path = 'G:\\Yehmh\\DNDF\\202309_DNDF\\plants_name.xlsx'

# Read coordinates CSV file
coord_df = pd.read_csv(coord_file_path)

for file in os.listdir(files_path):
    print(file)

    # Remove the file extension
    filename_without_extension = os.path.splitext(file)[0]
    coordinates = filename_without_extension.split("-")
    # If splitting by '-' didn't yield two parts, try splitting by '_'
    if len(coordinates) != 2:
        coordinates = filename_without_extension.split("_")
    
    plot = int(coordinates[0])
    transect = int(coordinates[1])
    print(plot, transect)

    df = pd.read_excel(os.path.join(files_path, file))
    
    for index, row in coord_df.iterrows():
        # Extract plot and line_num values
        coord_plot = row['Plot']
        coord_transect = row['Transect']

        # Check if the current row matches the plot and transect
        if coord_plot == plot and coord_transect == transect:
            # Calculate new columns using the current row's values
            df = df.assign(TWD97_X=row['TWD97_X'] + df['PLOTX'],
                           TWD97_Y=row['TWD97_Y'] + df['PLOTY'])
            # No need to continue the loop once the matching row is found
            break
    
    df['SP'] = df['Species_en'].astype(str)
    df.drop(columns=['Species_en', df.columns[0]], inplace=True)
     
    # Save the updated DataFrame to a new Excel file
    df.to_csv(f'G:/Yehmh/DNDF/tree_survey_coord_00/{plot}_{transect}.csv', index=False)


In [11]:
import math
import pandas as pd
import numpy as np

def calculate_angle(point1, point2):
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def rotate_point(point, angle_rad):
    # Perform rotation using rotation matrix
    rotated_x = point[0] * math.cos(angle_rad) - point[1] * math.sin(angle_rad)
    rotated_y = point[0] * math.sin(angle_rad) + point[1] * math.cos(angle_rad)
    return rotated_x, rotated_y

csv_file_path = 'G:/Yehmh/DNDF/tree_survey_coord_00/86_1.csv'
df = pd.read_csv(csv_file_path)

# Example points
point1 = (292328.6692, 2613020.622)
point2 = (292427.808, 2613025.95)

# Calculate angle between the line connecting the points and the horizontal line
angle_rad = math.atan2(point2[1] - point1[1], point2[0] - point1[0])

# Vectorized translation and rotation using pandas operations
x = df['TWD97_X'] - point1[0]
y = df['TWD97_Y'] - point1[1]
x_rotated, y_rotated = rotate_point((x, y), angle_rad)

# Update the dataframe with rotated coordinates and translate back
df['TWD97_X'] = x_rotated + point1[0]
df['TWD97_Y'] = y_rotated + point1[1]

# Save the modified dataframe to a new CSV file
df.to_csv(f'G:/Yehmh/DNDF/tree_survey_coord_rotated/86_1.csv', index=False)


In [24]:
import math
import pandas as pd
import numpy as np
import os

def calculate_angle(point1, point2):
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def rotate_point(point, angle_rad):
    # Perform rotation using rotation matrix
    rotated_x = point[0] * math.cos(angle_rad) - point[1] * math.sin(angle_rad)
    rotated_y = point[0] * math.sin(angle_rad) + point[1] * math.cos(angle_rad)
    return rotated_x, rotated_y

tree_survey_coord_folder = 'G:/2024_NDHU research assistant/DNDF/tree_survey_coord_00_dbh/'
plantline_coord_df = pd.read_csv('G:\\2024_NDHU research assistant\\DNDF\\DNDF_PlantLine_twd97_all.csv')
     
for filename in os.listdir(tree_survey_coord_folder):
    df = pd.read_csv(os.path.join(tree_survey_coord_folder, filename))
    filename_without_extension = os.path.splitext(filename)[0]
    plot, line_num = filename_without_extension.split("_")

    row1 = plantline_coord_df[(plantline_coord_df['Plot'] == int(plot)) & (plantline_coord_df['line_num'] == int(line_num)) & (plantline_coord_df['block_x'] == 0) & (plantline_coord_df['block_y'] == 0)]
    row2 = plantline_coord_df[(plantline_coord_df['Plot'] == int(plot)) & (plantline_coord_df['line_num'] == int(line_num)) & (plantline_coord_df['block_x'] == 5) & (plantline_coord_df['block_y'] == 0)]

    if row1.empty or row2.empty:
        print(f'No coordinates found for plot {plot}_{line_num}')
        continue

    point1 = (row1['TWD97_X'].values[0], row1['TWD97_Y'].values[0])
    point2 = (row2['TWD97_X'].values[0], row2['TWD97_Y'].values[0])

    # Calculate angle between the line connecting the points and the horizontal line
    angle_rad = math.atan2(point2[1] - point1[1], point2[0] - point1[0])

    # Vectorized translation and rotation using pandas operations
    x = df['TWD97_X'] - point1[0]
    y = df['TWD97_Y'] - point1[1]
    x_rotated, y_rotated = rotate_point((x, y), angle_rad)

    # Update the dataframe with rotated coordinates and translate back
    df['TWD97_X'] = x_rotated + point1[0]
    df['TWD97_Y'] = y_rotated + point1[1]

    # Save the modified dataframe to a new CSV file
    df.to_csv(f'G:/2024_NDHU research assistant/DNDF/tree_survey_coord_rotated_dbh/{plot}_{line_num}.csv', index=False)


No coordinates found for plot 108_1
No coordinates found for plot 108_2
No coordinates found for plot 115_1
No coordinates found for plot 115_2
No coordinates found for plot 123_1
No coordinates found for plot 127_1
No coordinates found for plot 133_1
No coordinates found for plot 133_2
No coordinates found for plot 134_1
No coordinates found for plot 146_1
No coordinates found for plot 146_2
No coordinates found for plot 14_1
No coordinates found for plot 150_1
No coordinates found for plot 156_1
No coordinates found for plot 20_1
No coordinates found for plot 20_2
No coordinates found for plot 24_1
No coordinates found for plot 24_2
No coordinates found for plot 25_1
No coordinates found for plot 26_1
No coordinates found for plot 29_1
No coordinates found for plot 52_1
No coordinates found for plot 52_2
No coordinates found for plot 56_1
No coordinates found for plot 65_1
No coordinates found for plot 65_2
No coordinates found for plot 66_1
No coordinates found for plot 66_2
No coor